# Yada Deconvolution

---



- Run the following cells for deconvolution using Yada.
- Example files are provided in data folder.
- In case that a special normalizations or scale is required edit the file yada.py.

## Import prerequisites

In [1]:
from IPython.display import FileLink, FileLinks
import pandas as pd


pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

!pip install tslearn
!rm -r Yada
!git clone https://github.com/zurkin1/Yada.git
!mv Yada/* .

     |████████████████████████████████| 778kB 5.3MB/s 
rm: cannot remove 'Yada': No such file or directory
Cloning into 'Yada'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 285 (delta 70), reused 47 (delta 18), pack-reused 159
Receiving objects: 100% (285/285), 53.76 MiB | 20.94 MiB/s, done.
Resolving deltas: 100% (103/103), done.


## Configure Input Files


In [2]:
from yada import *

#This is the mixture file in the format: columns: mix1, mix2, ..., rows: gene names.
mix = 'data/Abbas/mix.csv'
#Reference matrix name. Should be normalized as the mix data.
pure = 'data/Abbas/pure.csv'
labels = 'data/Abbas/labels.csv'

## Run deconvolution.

In [ ]:
#Select deconvolution methods and number of runs (or leave default values). Available methods: dtw_deconv, cibersort, dsection, lasso, nnls_deconv_constrained, ica_deconv (later).
result = run_deconv(10, mix, pure, dtw_deconv)
result

In [ ]:
#In case true proportions are available.
calc_corr(labels, result)

In [ ]:
#Download Result.
#FileLink('data/results.csv')
from google.colab import files
files.download('data/results.csv') 